<img src="../images/xarray_aws_zarr.png" />

# Plotting HRRR 2-meter temperatures

## Overview
1. Access archived HRRR data hosted on AWS in Zarr format
2. Visualize one of the variables (2m temperature) at an analysis time

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| Xarray Lessons 1-9| Necessary | |

* **Time to learn**: 30 minutes
***

## Imports

In [ ]:
import xarray as xr
import metpy
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import s3fs
from importlib.metadata import version

## What is Zarr?

Gridded datasets, especially those produced by operational meteorological centers such as NCEP and ECMWF, are typically in NetCDF and GRIB formats. Zarr is a relatively new data format. It is particularly relevant in the following two scenarios:
1. Datasets that are stored in what's called *object store*. This is a commonly-used storage method for cloud providers, such as Amazon, Google, and Microsoft.
2. Datasets that are typically too large to load into memory all at once.

Xarray supports the Zarr format in addition to NetCDF and GRIB. The [Pangeo](https://pangeo.io) project specifically recommends Zarr as the Xarray-amenable data format of choice in the cloud:
>
>"Our current preference for storing multidimensional array data in the cloud is the Zarr format. Zarr is a new storage format which, thanks to its simple yet well-designed specification, makes large datasets easily accessible to distributed computing. In Zarr datasets, the arrays are divided into chunks and compressed. These individual chunks can be stored as files on a filesystem or as objects in a cloud storage bucket. The metadata are stored in lightweight .json files. Zarr works well on both local filesystems and cloud-based object stores. Existing datasets can easily be converted to zarr via xarray’s zarr functions."


## Access archived HRRR data hosted on AWS in Zarr format</span>

For a number of years, the [Mesowest](https://mesowest.utah.edu/) group at the University of Utah has hosted an archive of data from NCEP's High Resolution Rapid Refresh model. This data, originally in GRIB-2 format, has been converted into Zarr and is freely available "in the cloud", on [Amazon Web Service's Simple Storage Service](https://aws.amazon.com/s3/), otherwise known as **S3**. Data is stored in S3 in a manner akin to (but different from) a Linux filesystem, using a [*bucket* and *object*](https://docs.aws.amazon.com/AmazonS3/latest/userguide/Welcome.html#CoreConcepts) model.

To interactively browse the contents of this archive, go to this link: [HRRRZarr File Browser on AWS](https://hrrrzarr.s3.amazonaws.com/index.html)

To access Zarr-formatted data stored in an S3 bucket, we follow a 3-step process:
1. Create URL(s) pointing to the bucket and object(s) that contain the data we want
1. Create *map(s)* to the object(s) with the **s3fs** library's `S3Map` method
1. Pass the *map(s)* to Xarray's `open_dataset` or `open_mfdataset` methods, and specify `zarr` as the format, via the `engine` argument.

<div class="alert alert-info">
A quirk in how these grids were converted from GRIB2 to Zarr means that the dimension variables are defined one directory up from where the data variables are. Thus, our strategy is to use Xarray's <code>open_mfdataset</code> method and pass in two AWS S3 file references to these two corresponding directories.
</div>

Create the URLs

In [ ]:
date = '20210214'
hour = '12'
var = 'TMP'
level = '2m_above_ground'
url1 = 's3://hrrrzarr/sfc/' + date + '/' + date + '_' + hour + 'z_anl.zarr/' + level + '/' + var + '/' + level
url2 = 's3://hrrrzarr/sfc/' + date + '/' + date + '_' + hour + 'z_anl.zarr/' + level + '/' + var

<div class="alert alert-info">
    In this case, <b>hrrrzarr</b> is the S3 <i>bucket</i> name. <b>2m_above_ground</b> and <b>TMP</b> are both <i>objects</i> within the <b>bucket</b>. The former object has the 2-meter temperature array, while the latter contains the coordinate arrays of the spatial dimensions of 2m temperature (i.e., x and y).
</div>

Connect to the S3 object store. With the [release of Zarr version 3](https://zarr.readthedocs.io/en/latest/user-guide/v3_migration.html), the methods to do this have changed from version 2 (see, e.g. Zarr issues [2706](https://github.com/zarr-developers/zarr-python/issues/2706) and [2748](https://github.com/zarr-developers/zarr-python/issues/2748)) . Test which version is installed and use the appropriate methodology.

In [ ]:
package = "zarr"
package_version = version(package)
major_version = int(package_version.split(".")[0])  # Extract the major version

if major_version == 3:      
    import zarr
    # strip leading 's3://' from url
    url1_3 = url1[5:]
    url2_3 = url2[5:]
    fs = s3fs.S3FileSystem(anon=True, asynchronous=True)
    file1 = zarr.storage.FsspecStore(fs, path=url1_3)
    file2 = zarr.storage.FsspecStore(fs, path=url2_3)
else:
    fs = s3fs.S3FileSystem(anon=True, asynchronous=False)
    file1 = s3fs.S3Map(url1, s3=fs)
    file2 = s3fs.S3Map(url2, s3=fs)

Use Xarray's `open_mfdataset` to create a `Dataset` from these two S3 objects.

In [ ]:
ds = xr.open_mfdataset([file1,file2], engine='zarr')

Examine the dataset.

In [ ]:
ds

Get the map projection for the HRRR, which is stored in the `grid` folder in the AWS bucket.

In [ ]:
HRRR_proj = pd.read_json ("https://hrrrzarr.s3.amazonaws.com/grid/projparams.json", orient="index")
HRRR_proj

In [ ]:
lat_0 = HRRR_proj.loc['lat_0'].astype('float32').squeeze()
lat_1 = HRRR_proj.loc['lat_1'].astype('float32').squeeze()
lat_2 = HRRR_proj.loc['lat_2'].astype('float32').squeeze()
lon_0 = HRRR_proj.loc['lon_0'].astype('float32').squeeze()
a = HRRR_proj.loc['a'].astype('float32').squeeze()
b = HRRR_proj.loc['b'].astype('float32').squeeze()

projData= ccrs.LambertConformal(central_longitude=lon_0, central_latitude=lat_0,
                                standard_parallels=[lat_1,lat_2],
                                globe=ccrs.Globe(semimajor_axis=a, semiminor_axis=b))

<div class="alert alert-warning"><b>Note: </b>
    The HRRR's projection assumes a <i>spherical earth</i>, whose semi-major/minor axes are both equal to 6371.229 km. We therefore need to explicitly define a <code>Globe</code> in Cartopy with these values.
</div>

Examine the dataset's coordinate variables. Each x- and y- value represents distance in meters from the central latitude and longitude.

In [ ]:
ds.coords

Create an object pointing to the dataset's data variable.

In [ ]:
airTemp = ds.TMP

When we examine the object, we see that it is a special type of `DataArray` ... a *[DaskArray](https://docs.dask.org/en/latest/array.html)*.

In [ ]:
airTemp

## Sidetrip: Dask

<div class="alert alert-info">Dask is a Python library that is especially well-suited for handling very large datasets (especially those that are too large to fit into RAM) and is nicely integrated with Xarray. We're going to defer a detailed exploration of Dask for now. But suffice it to say that when we use <code>open_mfdataset</code>, the resulting objects are <i>Dask</i> objects.
</div>

MetPy supports Dask arrays, and so performing a unit conversion is straightforward.

In [ ]:
airTemp = airTemp.metpy.convert_units('degC')

Verify that the object has the unit change

In [ ]:
airTemp

Similar to what we did for datasets whose projection-related coordinates were latitude and longitude, we define objects pointing to x and y now, so we can pass them to the plotting functions.

In [ ]:
x = airTemp.projection_x_coordinate
y = airTemp.projection_y_coordinate

## Visualize 2m temperature at an analysis time

First, just use Xarray's `plot` function to get a quick look to verify that things look right.

In [ ]:
airTemp.plot(figsize=(11,8.5))

To facilitate the bounds of the contour intervals, obtain the min and max values from this `DataArray`.

<div class="alert alert-warning">
    A Dask array is even more <i>lazy</i> in terms of its data loading than a basic <code>DataArray</code> in Xarray. If we want to perform a computation on this array, e.g. calculate the mean, min, or max, note that we don't get a result straightaway ... we get another Dask array.
</div>

In [ ]:
airTemp.min()

<div class="alert alert-info">
With Dask arrays, applying the min and max functions doesn't actually do the computation ... instead, it is creating a <i>task graph</i> which describes how the computations would be launched. You will need to call Dask's <code>compute</code> function to actually trigger the computation.   
</div>

In [ ]:
minTemp = airTemp.min().compute()
maxTemp = airTemp.max().compute()

In [ ]:
minTemp.values, maxTemp.values

Based on the min and max, define a range of values used for contouring. Let's invoke NumPy's `floor` and `ceil`(ing) functions so these values conform to whatever variable we are contouring.

In [ ]:
fint = np.arange(np.floor(minTemp.values),np.ceil(maxTemp.values) + 2, 2)

In [ ]:
fint

<div class="alert alert-warning">
 For a single map, setting the contour fill values as we did above is appropriate. But if you were producing a series of maps that span a range of times, a consistent (and thus wider) range of values would be better.
</div>

## Plot the map
We'll define the plot extent to nicely encompass the HRRR's spatial domain.

In [ ]:
latN = 50.4
latS = 24.25
lonW = -123.8
lonE = -71.2

res = '50m'

fig = plt.figure(figsize=(18,12))
ax = fig.add_subplot(1,1,1,projection=projData)
ax.set_extent ([lonW,lonE,latS,latN],crs=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale(res))
ax.add_feature(cfeature.STATES.with_scale(res))

# Add the title
tl1 = 'HRRR 2m temperature (°C)'
tl2 = f'Analysis valid at: {hour}00 UTC {date}'  
ax.set_title(f'{tl1}\n{tl2}',fontsize=16)
# Contour fill
CF = ax.contourf(x,y,airTemp,levels=fint,cmap=plt.get_cmap('coolwarm'))
# Make a colorbar for the ContourSet returned by the contourf call.
cbar = fig.colorbar(CF,shrink=0.5)
cbar.set_label(f'2m Temperature (°C)', size='large')

---
## Summary
* Xarray can read gridded datasets in Zarr format, which is ideal for a cloud-based object store system such as S3.
* Xarray and MetPy both support *Dask*, a library that is particularly well-suited for very large datasets.

## What's next?
On your own, browse the [**hrrrzarr**](https://hrrrzarr.s3.amazonaws.com/index.html) S3 bucket. Try making maps for different variables and/or different times.

## Resources and References

1. [HRRR in Zarr format](https://mesowest.utah.edu/html/hrrr/)
1. [NCEP's HRRR S3 archive (GRIB format)](https://registry.opendata.aws/noaa-hrrr-pds/)
1. [What is *object store*?](https://docs.aws.amazon.com/AmazonS3/latest/userguide/Welcome.html)
1. [Xarray's Dask implementation](http://xarray.pydata.org/en/stable/user-guide/dask.html)